In [ ]:
# Configuración del entorno
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

# Funciones helper
def format_currency(value):
    """Formatear valores monetarios"""
    if pd.isna(value):
        return 'N/A'
    if value >= 1e9:
        return f"${value/1e9:.2f}B"
    elif value >= 1e6:
        return f"${value/1e6:.2f}M"
    elif value >= 1e3:
        return f"${value/1e3:.2f}K"
    else:
        return f"${value:.2f}"

def format_number(value):
    """Formatear números grandes"""
    if pd.isna(value):
        return 'N/A'
    if value >= 1e6:
        return f"{value/1e6:.1f}M"
    elif value >= 1e3:
        return f"{value/1e3:.1f}K"
    else:
        return f"{value:,.0f}"

print("🔧 Entorno configurado para análisis de outliers")
print("📊 Librerías cargadas: pandas, numpy, matplotlib, seaborn, scipy")


# 🔍 **Análisis de Outliers y Casos Excepcionales**
## **Identificación de Patrones Anómalos en Datos Hospitalarios**

---

<div style="background-color: #f8f9fa; padding: 20px; border-left: 5px solid #dc3545; margin: 20px 0;">

### 📋 **Información del Documento**

| **Campo** | **Valor** |
|-----------|-----------|
| **Proyecto** | Economía Salud - UCDS INER |
| **Fecha** | 7 de Enero, 2025 |
| **Autor** | Equipo de Ciencia de Datos |
| **Dataset** | `resultados_pacientes_estandarizados.csv` |
| **Tipo** | Análisis de Outliers y Anomalías |
| **Versión** | 2.0 - Análisis Especializado |

</div>

---

### 🎯 **Objetivos del Análisis de Outliers**

1. **🔍 Identificación de Casos Extremos** en costos, cantidades y duración de tratamiento
2. **📊 Análisis de Distribuciones** para detectar patrones anómalos
3. **💰 Evaluación Financiera** de casos de alto costo y su impacto
4. **🏥 Caracterización Clínica** de pacientes con comportamientos atípicos
5. **⚠️ Detección de Errores** potenciales en el registro de datos
6. **🚀 Identificación de Oportunidades** de optimización y control de costos

---

### 📊 **Resumen de Hallazgos de Outliers**

<div style="background-color: #fff3cd; padding: 15px; border-radius: 8px; margin: 15px 0; border-left: 4px solid #ffc107;">

#### **⚠️ Casos Críticos Identificados**
- **Pacientes de Ultra-Alto Costo:** >$10M por paciente
- **Cargos Masivos:** >10,000 unidades en un solo cargo
- **Anomalías Temporales:** Estancias extremadamente prolongadas
- **Servicios Atípicos:** Costos por unidad >$50,000

</div>

---


In [1]:
# =========================================================================
# CONFIGURACIÓN DEL ENTORNO PARA ANÁLISIS DE OUTLIERS
# =========================================================================

# Librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.patches as mpatches

# Configuración general
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', 100)

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Colores para outliers
OUTLIER_COLORS = {
    'extreme': '#dc3545',     # Rojo para casos extremos
    'moderate': '#ffc107',    # Amarillo para casos moderados  
    'normal': '#28a745',      # Verde para casos normales
    'suspicious': '#6f42c1',  # Púrpura para casos sospechosos
    'primary': '#007bff'      # Azul para referencia
}

# Funciones de utilidad
def format_number(num):
    """Formatea números grandes con comas y sufijos apropiados"""
    if pd.isna(num):
        return "N/A"
    if num >= 1_000_000:
        return f"{num/1_000_000:.1f}M"
    elif num >= 1_000:
        return f"{num/1_000:.0f}K"
    else:
        return f"{num:,.0f}"

def format_currency(amount):
    """Formatea montos como moneda"""
    if pd.isna(amount):
        return "N/A"
    if amount >= 1_000_000_000:
        return f"${amount/1_000_000_000:.2f}B"
    elif amount >= 1_000_000:
        return f"${amount/1_000_000:.1f}M"
    elif amount >= 1_000:
        return f"${amount/1_000:.0f}K"
    else:
        return f"${amount:,.2f}"

def detect_outliers_iqr(data, multiplier=1.5):
    """Detecta outliers usando el método IQR"""
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return (data < lower_bound) | (data > upper_bound)

def detect_outliers_zscore(data, threshold=3):
    """Detecta outliers usando Z-score"""
    z_scores = np.abs(stats.zscore(data, nan_policy='omit'))
    return z_scores > threshold

def detect_outliers_modified_zscore(data, threshold=3.5):
    """Detecta outliers usando Modified Z-score (más robusto)"""
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    modified_z_scores = 0.6745 * (data - median) / mad
    return np.abs(modified_z_scores) > threshold

print("✅ Configuración de análisis de outliers completada")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 Numpy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")
print(f"⏰ Análisis iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 Métodos de detección configurados: IQR, Z-score, Modified Z-score")


✅ Configuración de análisis de outliers completada
📊 Pandas version: 2.3.0
🔢 Numpy version: 1.26.4
📈 Matplotlib version: 3.10.3
🎨 Seaborn version: 0.13.2
⏰ Análisis iniciado: 2025-07-07 12:26:50
🎯 Métodos de detección configurados: IQR, Z-score, Modified Z-score


In [2]:
# =========================================================================
# CARGA DE DATOS Y PREPARACIÓN PARA ANÁLISIS DE OUTLIERS
# =========================================================================

print("="*80)
print("📂 CARGANDO DATASET PARA ANÁLISIS DE OUTLIERS")
print("="*80)

# Cargar dataset
try:
    df = pd.read_csv('../../data/processed/resultados_pacientes_estandarizados.csv', 
                     low_memory=False)
    print("✅ Dataset cargado exitosamente")
except FileNotFoundError:
    print("❌ Error: No se pudo encontrar el archivo de datos")
    raise
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    raise

# Información básica del dataset
print(f"\n📏 **DIMENSIONES DEL DATASET**")
print(f"   • Filas: {df.shape[0]:,}")
print(f"   • Columnas: {df.shape[1]}")

# Crear columnas calculadas para análisis de outliers
df['costo_por_unidad'] = df['monto_nivel_6'] / df['cantidad']
df['costo_por_unidad'] = df['costo_por_unidad'].replace([np.inf, -np.inf], np.nan)

print(f"\n🔧 **PREPARACIÓN DE DATOS PARA OUTLIERS**")
print(f"   • Columna 'costo_por_unidad' calculada")
print(f"   • Valores infinitos reemplazados por NaN")

# Estadísticas básicas de las variables clave para outliers
variables_outliers = ['cantidad', 'monto_nivel_6', 'costo_por_unidad']

print(f"\n📊 **ESTADÍSTICAS BÁSICAS DE VARIABLES CLAVE**")
print("-" * 80)
print(f"{'VARIABLE':<20} {'MIN':<15} {'MAX':<15} {'MEAN':<15} {'STD':<15} {'NULLS':<10}")
print("-" * 80)

for var in variables_outliers:
    if var in df.columns:
        min_val = df[var].min()
        max_val = df[var].max()
        mean_val = df[var].mean()
        std_val = df[var].std()
        nulls = df[var].isnull().sum()
        
        print(f"{var:<20} {min_val:<15,.2f} {max_val:<15,.2f} {mean_val:<15,.2f} {std_val:<15,.2f} {nulls:<10,}")

# Crear resumen por paciente para análisis
print(f"\n👥 **CREANDO RESUMEN POR PACIENTE**")
paciente_summary = df.groupby('paciente').agg({
    'paciente': 'count',           # número de cargos
    'cantidad': 'sum',             # total de unidades
    'monto_nivel_6': 'sum',        # costo total
    'costo_por_unidad': 'mean',    # costo promedio por unidad
    'origen': lambda x: x.nunique(),  # número de orígenes diferentes
    'area_servicio': lambda x: x.nunique()  # número de áreas diferentes
}).round(2)

paciente_summary.columns = ['total_cargos', 'total_unidades', 'costo_total', 
                           'costo_promedio_unidad', 'num_origenes', 'num_areas']

print(f"   • Resumen por paciente creado: {len(paciente_summary):,} pacientes")
print(f"   • Variables calculadas: cargos, unidades, costos, diversidad de servicios")

# Vista previa del resumen por paciente
print(f"\n👀 **VISTA PREVIA RESUMEN POR PACIENTE**")
print("-" * 80)
display(paciente_summary.head())


📂 CARGANDO DATASET PARA ANÁLISIS DE OUTLIERS
✅ Dataset cargado exitosamente

📏 **DIMENSIONES DEL DATASET**
   • Filas: 2,399,200
   • Columnas: 18

🔧 **PREPARACIÓN DE DATOS PARA OUTLIERS**
   • Columna 'costo_por_unidad' calculada
   • Valores infinitos reemplazados por NaN

📊 **ESTADÍSTICAS BÁSICAS DE VARIABLES CLAVE**
--------------------------------------------------------------------------------
VARIABLE             MIN             MAX             MEAN            STD             NULLS     
--------------------------------------------------------------------------------
cantidad             0.00            62,345.00       5.51            132.10          0         
monto_nivel_6        0.00            88,013,804.29   491.31          57,225.76       0         
costo_por_unidad     0.00            198,069.00      382.37          2,352.77        25        

👥 **CREANDO RESUMEN POR PACIENTE**
   • Resumen por paciente creado: 5,782 pacientes
   • Variables calculadas: cargos, unidades, c

,total_cargos,total_unidades,costo_total,costo_promedio_unidad,num_origenes,num_areas
paciente,,,,,,
327,72,336.00,37399.37,479.51,2,11
1905,2,2.00,63.82,31.91,1,2
2079,322,1820.00,188129.08,546.06,3,17
3939,78,368.00,34679.82,389.86,2,12
5400,43,140.00,22877.14,490.78,1,10


---

## 🎯 **1. DETECCIÓN DE OUTLIERS POR COSTO TOTAL**

En esta sección identificamos **pacientes con costos extremadamente altos** que pueden representar casos clínicos complejos o errores en el registro.

<div style="background-color: #f8d7da; padding: 15px; border-left: 4px solid #dc3545; margin: 15px 0;">
    <strong>⚠️ Criterios de Detección:</strong><br>
    • <strong>IQR Method:</strong> Valores fuera de Q3 + 1.5*IQR<br>
    • <strong>Z-Score:</strong> |Z| > 3 (casos extremos)<br>
    • <strong>Modified Z-Score:</strong> |MAD-Z| > 3.5 (más robusto a outliers)
</div>


In [3]:
# =========================================================================
# ANÁLISIS DE OUTLIERS POR COSTO TOTAL DE PACIENTES
# =========================================================================

print("="*80)
print("💰 DETECCIÓN DE OUTLIERS POR COSTO TOTAL")
print("="*80)

# Variable objetivo: costo total por paciente
costo_data = paciente_summary['costo_total'].dropna()

# Aplicar diferentes métodos de detección
outliers_iqr = detect_outliers_iqr(costo_data, multiplier=1.5)
outliers_zscore = detect_outliers_zscore(costo_data, threshold=3)
outliers_modified_zscore = detect_outliers_modified_zscore(costo_data, threshold=3.5)

# Combinar resultados
paciente_summary['outlier_iqr_costo'] = outliers_iqr
paciente_summary['outlier_zscore_costo'] = outliers_zscore
paciente_summary['outlier_mod_zscore_costo'] = outliers_modified_zscore

# Crear categorías de outliers
def categorize_outlier_severity(row):
    methods_count = sum([row['outlier_iqr_costo'], row['outlier_zscore_costo'], row['outlier_mod_zscore_costo']])
    if methods_count >= 3:
        return 'extreme'
    elif methods_count >= 2:
        return 'moderate'
    elif methods_count >= 1:
        return 'mild'
    else:
        return 'normal'

paciente_summary['outlier_severity_costo'] = paciente_summary.apply(categorize_outlier_severity, axis=1)

# Estadísticas de detección
print("📊 **RESULTADOS DE DETECCIÓN DE OUTLIERS POR COSTO**")
print("-" * 60)

methods = ['IQR', 'Z-Score', 'Modified Z-Score']
columns = ['outlier_iqr_costo', 'outlier_zscore_costo', 'outlier_mod_zscore_costo']

for method, col in zip(methods, columns):
    count = paciente_summary[col].sum()
    percentage = (count / len(paciente_summary)) * 100
    print(f"   • {method:<20}: {count:>4,} outliers ({percentage:>5.2f}%)")

# Distribución por severidad
print(f"\n🎯 **DISTRIBUCIÓN POR SEVERIDAD DE OUTLIERS**")
print("-" * 60)
severity_counts = paciente_summary['outlier_severity_costo'].value_counts()
for severity, count in severity_counts.items():
    percentage = (count / len(paciente_summary)) * 100
    print(f"   • {severity.capitalize():<15}: {count:>4,} pacientes ({percentage:>5.2f}%)")

# Análisis de casos extremos (outliers en los 3 métodos)
extreme_outliers = paciente_summary[paciente_summary['outlier_severity_costo'] == 'extreme'].copy()

if len(extreme_outliers) > 0:
    print(f"\n🚨 **CASOS EXTREMOS EN COSTO (TOP 10)**")
    print("-" * 90)
    print(f"{'PACIENTE':<12} {'COSTO TOTAL':<15} {'CARGOS':<8} {'UNIDADES':<12} {'COST/UNIDAD':<12} {'AREAS':<8}")
    print("-" * 90)
    
    extreme_outliers_sorted = extreme_outliers.nlargest(10, 'costo_total')
    for idx, (paciente, row) in enumerate(extreme_outliers_sorted.iterrows(), 1):
        print(f"{paciente:<12} {format_currency(row['costo_total']):<15} {row['total_cargos']:<8,.0f} "
              f"{row['total_unidades']:<12,.0f} {format_currency(row['costo_promedio_unidad']):<12} {row['num_areas']:<8}")

# Análisis del impacto financiero de outliers
print(f"\n💡 **IMPACTO FINANCIERO DE OUTLIERS EN COSTO**")
print("-" * 60)

total_costo = paciente_summary['costo_total'].sum()
outliers_all = paciente_summary[paciente_summary['outlier_severity_costo'].isin(['extreme', 'moderate', 'mild'])]
costo_outliers = outliers_all['costo_total'].sum()
impacto_porcentual = (costo_outliers / total_costo) * 100

print(f"   • Costo total dataset: {format_currency(total_costo)}")
print(f"   • Costo outliers: {format_currency(costo_outliers)}")
print(f"   • Porcentaje del costo total: {impacto_porcentual:.2f}%")
print(f"   • Número de outliers: {len(outliers_all):,} de {len(paciente_summary):,} pacientes")

# Estadísticas descriptivas por categoría
print(f"\n📈 **ESTADÍSTICAS POR CATEGORÍA DE OUTLIERS**")
print("-" * 80)
print(f"{'CATEGORÍA':<15} {'COUNT':<8} {'COSTO_PROM':<15} {'COSTO_MED':<15} {'CARGOS_PROM':<12}")
print("-" * 80)

for categoria in ['normal', 'mild', 'moderate', 'extreme']:
    subset = paciente_summary[paciente_summary['outlier_severity_costo'] == categoria]
    if len(subset) > 0:
        count = len(subset)
        costo_mean = subset['costo_total'].mean()
        costo_median = subset['costo_total'].median()
        cargos_mean = subset['total_cargos'].mean()
        print(f"{categoria.capitalize():<15} {count:<8,} {format_currency(costo_mean):<15} "
              f"{format_currency(costo_median):<15} {cargos_mean:<12,.1f}")

print(f"\n✅ Análisis de outliers por costo completado")


💰 DETECCIÓN DE OUTLIERS POR COSTO TOTAL
📊 **RESULTADOS DE DETECCIÓN DE OUTLIERS POR COSTO**
------------------------------------------------------------
   • IQR                 :  526 outliers ( 9.10%)
   • Z-Score             :    8 outliers ( 0.14%)
   • Modified Z-Score    :  577 outliers ( 9.98%)

🎯 **DISTRIBUCIÓN POR SEVERIDAD DE OUTLIERS**
------------------------------------------------------------
   • Normal         : 5,205 pacientes (90.02%)
   • Moderate       :  518 pacientes ( 8.96%)
   • Mild           :   51 pacientes ( 0.88%)
   • Extreme        :    8 pacientes ( 0.14%)

🚨 **CASOS EXTREMOS EN COSTO (TOP 10)**
------------------------------------------------------------------------------------------
PACIENTE     COSTO TOTAL     CARGOS   UNIDADES     COST/UNIDAD  AREAS   
------------------------------------------------------------------------------------------
599267       $88.3M          711      22,480       $322.99      18      
136738       $5.6M           4,096   

---

## 📦 **2. DETECCIÓN DE OUTLIERS POR CANTIDAD/UNIDADES**

Identificamos **cargos individuales con cantidades extremadamente altas** que pueden indicar errores de captura o casos clínicos especiales.

<div style="background-color: #d1ecf1; padding: 15px; border-left: 4px solid #17a2b8; margin: 15px 0;">
    <strong>🔍 Enfoques de Análisis:</strong><br>
    • <strong>Cargos Individuales:</strong> Cantidades anómalas por registro<br>
    • <strong>Pacientes:</strong> Totales de unidades por paciente<br>
    • <strong>Por Origen:</strong> Patrones específicos por servicio
</div>


In [4]:
# =========================================================================
# ANÁLISIS DE OUTLIERS POR CANTIDAD/UNIDADES
# =========================================================================

print("="*80)
print("📦 DETECCIÓN DE OUTLIERS POR CANTIDAD/UNIDADES")
print("="*80)

# 1. ANÁLISIS DE CARGOS INDIVIDUALES CON CANTIDADES EXTREMAS
print("🔍 **1. OUTLIERS EN CARGOS INDIVIDUALES**")
print("-" * 60)

cantidad_data = df['cantidad'].dropna()

# Aplicar métodos de detección a cantidades
outliers_cantidad_iqr = detect_outliers_iqr(cantidad_data, multiplier=1.5)
outliers_cantidad_zscore = detect_outliers_zscore(cantidad_data, threshold=3)
outliers_cantidad_modified = detect_outliers_modified_zscore(cantidad_data, threshold=3.5)

# Añadir flags al dataframe original
df['outlier_cantidad_iqr'] = outliers_cantidad_iqr
df['outlier_cantidad_zscore'] = outliers_cantidad_zscore
df['outlier_cantidad_modified'] = outliers_cantidad_modified

# Estadísticas de detección en cargos individuales
methods = ['IQR', 'Z-Score', 'Modified Z-Score']
columns = ['outlier_cantidad_iqr', 'outlier_cantidad_zscore', 'outlier_cantidad_modified']

for method, col in zip(methods, columns):
    count = df[col].sum()
    percentage = (count / len(df)) * 100
    print(f"   • {method:<20}: {count:>8,} cargos outliers ({percentage:>5.2f}%)")

# Top 10 cargos con mayor cantidad
print(f"\n🚨 **TOP 10 CARGOS CON MAYOR CANTIDAD**")
print("-" * 100)
print(f"{'PACIENTE':<12} {'CANTIDAD':<12} {'COSTO':<15} {'ORIGEN':<15} {'AREA_SERVICIO':<25} {'DESCRIPCION':<20}")
print("-" * 100)

top_cantidad = df.nlargest(10, 'cantidad')
for _, row in top_cantidad.iterrows():
    descripcion = str(row['descripcion'])[:18] + '..' if len(str(row['descripcion'])) > 20 else str(row['descripcion'])
    area = str(row['area_servicio'])[:23] + '..' if len(str(row['area_servicio'])) > 25 else str(row['area_servicio'])
    print(f"{row['paciente']:<12} {row['cantidad']:<12,.0f} {format_currency(row['monto_nivel_6']):<15} "
          f"{row['origen']:<15} {area:<25} {descripcion:<20}")

# 2. ANÁLISIS POR ORIGEN DE SERVICIO
print(f"\n🏥 **2. OUTLIERS POR ORIGEN DE SERVICIO**")
print("-" * 60)

for origen in df['origen'].unique():
    df_origen = df[df['origen'] == origen]
    cantidad_origen = df_origen['cantidad']
    
    # Estadísticas básicas
    Q1 = cantidad_origen.quantile(0.25)
    Q3 = cantidad_origen.quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_count = (cantidad_origen > upper_bound).sum()
    max_cantidad = cantidad_origen.max()
    mean_cantidad = cantidad_origen.mean()
    
    print(f"\n   🔸 {origen}:")
    print(f"      • Outliers (IQR): {outliers_count:,} de {len(df_origen):,} cargos ({outliers_count/len(df_origen)*100:.2f}%)")
    print(f"      • Cantidad máxima: {max_cantidad:,.0f}")
    print(f"      • Cantidad promedio: {mean_cantidad:.2f}")
    print(f"      • Umbral outlier (Q3+1.5*IQR): {upper_bound:.2f}")

# 3. ANÁLISIS DE PACIENTES CON ALTA CANTIDAD TOTAL
print(f"\n👥 **3. PACIENTES CON MAYOR TOTAL DE UNIDADES**")
print("-" * 80)

# Aplicar detección de outliers a totales por paciente
unidades_data = paciente_summary['total_unidades'].dropna()
outliers_unidades_iqr = detect_outliers_iqr(unidades_data, multiplier=1.5)
outliers_unidades_zscore = detect_outliers_zscore(unidades_data, threshold=3)

paciente_summary['outlier_unidades_iqr'] = outliers_unidades_iqr
paciente_summary['outlier_unidades_zscore'] = outliers_unidades_zscore

print("📊 **Outliers en Total de Unidades por Paciente:**")
print(f"   • IQR Method: {outliers_unidades_iqr.sum():,} pacientes ({outliers_unidades_iqr.sum()/len(paciente_summary)*100:.2f}%)")
print(f"   • Z-Score Method: {outliers_unidades_zscore.sum():,} pacientes ({outliers_unidades_zscore.sum()/len(paciente_summary)*100:.2f}%)")

# Top 10 pacientes con más unidades totales
print(f"\n🔝 **TOP 10 PACIENTES CON MÁS UNIDADES TOTALES**")
print("-" * 90)
print(f"{'PACIENTE':<12} {'TOTAL_UNIDADES':<15} {'TOTAL_CARGOS':<12} {'COSTO_TOTAL':<15} {'PROM_UNID/CARGO':<15}")
print("-" * 90)

top_unidades = paciente_summary.nlargest(10, 'total_unidades')
for paciente, row in top_unidades.iterrows():
    unidades_por_cargo = row['total_unidades'] / row['total_cargos'] if row['total_cargos'] > 0 else 0
    print(f"{paciente:<12} {row['total_unidades']:<15,.0f} {row['total_cargos']:<12,.0f} "
          f"{format_currency(row['costo_total']):<15} {unidades_por_cargo:<15.2f}")

# 4. ANÁLISIS DE DISTRIBUCIÓN DE CANTIDADES EXTREMAS
print(f"\n📈 **4. DISTRIBUCIÓN DE CANTIDADES EXTREMAS**")
print("-" * 60)

# Definir rangos de cantidad
rangos = [
    (1, 10, "Normal (1-10)"),
    (11, 100, "Elevado (11-100)"),
    (101, 1000, "Alto (101-1000)"),
    (1001, 10000, "Muy Alto (1001-10K)"),
    (10001, float('inf'), "Extremo (>10K)")
]

print(f"{'RANGO':<20} {'CARGOS':<12} {'%_CARGOS':<10} {'UNIDADES':<15} {'%_UNIDADES':<12} {'COSTO_PROM':<15}")
print("-" * 90)

for min_val, max_val, label in rangos:
    if max_val == float('inf'):
        mask = df['cantidad'] >= min_val
    else:
        mask = (df['cantidad'] >= min_val) & (df['cantidad'] <= max_val)
    
    subset = df[mask]
    if len(subset) > 0:
        count_cargos = len(subset)
        pct_cargos = count_cargos / len(df) * 100
        total_unidades = subset['cantidad'].sum()
        pct_unidades = total_unidades / df['cantidad'].sum() * 100
        costo_prom = subset['monto_nivel_6'].mean()
        
        print(f"{label:<20} {count_cargos:<12,} {pct_cargos:<10.2f} {total_unidades:<15,.0f} "
              f"{pct_unidades:<12.2f} {format_currency(costo_prom):<15}")

print(f"\n✅ Análisis de outliers por cantidad completado")


📦 DETECCIÓN DE OUTLIERS POR CANTIDAD/UNIDADES
🔍 **1. OUTLIERS EN CARGOS INDIVIDUALES**
------------------------------------------------------------
   • IQR                 :  274,376 cargos outliers (11.44%)
   • Z-Score             :      163 cargos outliers ( 0.01%)
   • Modified Z-Score    : 1,060,390 cargos outliers (44.20%)

🚨 **TOP 10 CARGOS CON MAYOR CANTIDAD**
----------------------------------------------------------------------------------------------------
PACIENTE     CANTIDAD     COSTO           ORIGEN          AREA_SERVICIO             DESCRIPCION         
----------------------------------------------------------------------------------------------------
674567       62,345       $3.0M           Hospitalización MATERIAL QUIRÚRGICO       BATA DESECHABLE NO..
708594       51,255       $52K            Hospitalización MEDICINAS Y PRODUC. FA    GUANTE P/EXP.  AMB..
714823       40,259       $72K            Hospitalización MATERIALES E INSUMOS      GUANTE P/ EXPLORAC..
696148

---

## 💰 **3. DETECCIÓN DE OUTLIERS POR COSTO POR UNIDAD**

Identificamos **servicios con costos unitarios extremadamente altos** que pueden indicar servicios especializados, errores de facturación o casos clínicos complejos.

<div style="background-color: #d4edda; padding: 15px; border-left: 4px solid #28a745; margin: 15px 0;">
    <strong>💡 Valor Estratégico:</strong><br>
    • <strong>Control de Costos:</strong> Identificar servicios con pricing anómalo<br>
    • <strong>Auditoría:</strong> Detectar posibles errores de facturación<br>
    • <strong>Optimización:</strong> Oportunidades de negociación con proveedores
</div>


In [5]:
# =========================================================================
# ANÁLISIS DE OUTLIERS POR COSTO POR UNIDAD
# =========================================================================

print("="*80)
print("💰 DETECCIÓN DE OUTLIERS POR COSTO POR UNIDAD")
print("="*80)

# Preparar datos de costo por unidad (eliminar NaN e infinitos)
costo_unidad_data = df['costo_por_unidad'].dropna()
costo_unidad_data = costo_unidad_data[costo_unidad_data != np.inf]
costo_unidad_data = costo_unidad_data[costo_unidad_data != -np.inf]

print(f"📊 **Datos válidos para análisis:** {len(costo_unidad_data):,} de {len(df):,} registros")

# 1. DETECCIÓN GLOBAL DE OUTLIERS EN COSTO POR UNIDAD
print(f"\n🔍 **1. OUTLIERS GLOBALES EN COSTO POR UNIDAD**")
print("-" * 60)

# Aplicar métodos de detección
outliers_costo_unidad_iqr = detect_outliers_iqr(costo_unidad_data, multiplier=1.5)
outliers_costo_unidad_zscore = detect_outliers_zscore(costo_unidad_data, threshold=3)
outliers_costo_unidad_modified = detect_outliers_modified_zscore(costo_unidad_data, threshold=3.5)

# Crear máscaras para el dataframe completo
df['outlier_costo_unidad_iqr'] = False
df['outlier_costo_unidad_zscore'] = False
df['outlier_costo_unidad_modified'] = False

# Aplicar máscaras solo a registros válidos
valid_mask = df['costo_por_unidad'].notna() & (df['costo_por_unidad'] != np.inf) & (df['costo_por_unidad'] != -np.inf)
df.loc[valid_mask, 'outlier_costo_unidad_iqr'] = outliers_costo_unidad_iqr
df.loc[valid_mask, 'outlier_costo_unidad_zscore'] = outliers_costo_unidad_zscore
df.loc[valid_mask, 'outlier_costo_unidad_modified'] = outliers_costo_unidad_modified

# Estadísticas de detección
methods = ['IQR', 'Z-Score', 'Modified Z-Score']
columns = ['outlier_costo_unidad_iqr', 'outlier_costo_unidad_zscore', 'outlier_costo_unidad_modified']

for method, col in zip(methods, columns):
    count = df[col].sum()
    percentage = (count / len(df)) * 100
    print(f"   • {method:<20}: {count:>8,} cargos outliers ({percentage:>5.2f}%)")

# 2. TOP CARGOS CON MAYOR COSTO POR UNIDAD
print(f"\n🚨 **2. TOP 15 CARGOS CON MAYOR COSTO POR UNIDAD**")
print("-" * 120)
print(f"{'PACIENTE':<12} {'COSTO/UNIDAD':<15} {'CANTIDAD':<10} {'COSTO_TOTAL':<15} {'ORIGEN':<15} {'AREA_SERVICIO':<25} {'DESCRIPCION':<25}")
print("-" * 120)

top_costo_unidad = df[valid_mask].nlargest(15, 'costo_por_unidad')
for _, row in top_costo_unidad.iterrows():
    descripcion = str(row['descripcion'])[:23] + '..' if len(str(row['descripcion'])) > 25 else str(row['descripcion'])
    area = str(row['area_servicio'])[:23] + '..' if len(str(row['area_servicio'])) > 25 else str(row['area_servicio'])
    print(f"{row['paciente']:<12} {format_currency(row['costo_por_unidad']):<15} {row['cantidad']:<10,.0f} "
          f"{format_currency(row['monto_nivel_6']):<15} {row['origen']:<15} {area:<25} {descripcion:<25}")

# 3. ANÁLISIS POR ORIGEN DE SERVICIO
print(f"\n🏥 **3. OUTLIERS POR ORIGEN DE SERVICIO EN COSTO/UNIDAD**")
print("-" * 80)

for origen in df['origen'].unique():
    df_origen = df[(df['origen'] == origen) & valid_mask]
    if len(df_origen) > 0:
        costo_unidad_origen = df_origen['costo_por_unidad']
        
        # Estadísticas básicas
        Q1 = costo_unidad_origen.quantile(0.25)
        Q3 = costo_unidad_origen.quantile(0.75)
        IQR = Q3 - Q1
        upper_bound = Q3 + 1.5 * IQR
        
        outliers_count = (costo_unidad_origen > upper_bound).sum()
        max_costo = costo_unidad_origen.max()
        median_costo = costo_unidad_origen.median()
        mean_costo = costo_unidad_origen.mean()
        
        print(f"\n   🔸 {origen}:")
        print(f"      • Outliers (IQR): {outliers_count:,} de {len(df_origen):,} cargos ({outliers_count/len(df_origen)*100:.2f}%)")
        print(f"      • Costo/unidad máximo: {format_currency(max_costo)}")
        print(f"      • Costo/unidad promedio: {format_currency(mean_costo)}")
        print(f"      • Costo/unidad mediana: {format_currency(median_costo)}")
        print(f"      • Umbral outlier: {format_currency(upper_bound)}")

# 4. ANÁLISIS DE DISTRIBUCIÓN DE COSTOS POR UNIDAD
print(f"\n📈 **4. DISTRIBUCIÓN DE COSTOS POR UNIDAD**")
print("-" * 70)

# Definir rangos de costo por unidad
rangos_costo = [
    (0, 100, "Bajo (≤$100)"),
    (101, 500, "Medio ($101-$500)"),
    (501, 1000, "Alto ($501-$1K)"),
    (1001, 5000, "Muy Alto ($1K-$5K)"),
    (5001, 10000, "Extremo ($5K-$10K)"),
    (10001, float('inf'), "Ultra-Extremo (>$10K)")
]

print(f"{'RANGO':<25} {'CARGOS':<12} {'%_CARGOS':<10} {'COSTO_TOTAL':<15} {'%_COSTO':<10}")
print("-" * 75)

total_costo_valido = df[valid_mask]['monto_nivel_6'].sum()

for min_val, max_val, label in rangos_costo:
    if max_val == float('inf'):
        mask = (df['costo_por_unidad'] >= min_val) & valid_mask
    else:
        mask = (df['costo_por_unidad'] >= min_val) & (df['costo_por_unidad'] <= max_val) & valid_mask
    
    subset = df[mask]
    if len(subset) > 0:
        count_cargos = len(subset)
        pct_cargos = count_cargos / len(df[valid_mask]) * 100
        total_costo = subset['monto_nivel_6'].sum()
        pct_costo = total_costo / total_costo_valido * 100
        
        print(f"{label:<25} {count_cargos:<12,} {pct_cargos:<10.2f} {format_currency(total_costo):<15} {pct_costo:<10.2f}")

# 5. SERVICIOS ESPECÍFICOS CON COSTOS ANÓMALOS
print(f"\n🎯 **5. SERVICIOS MÁS FRECUENTES EN OUTLIERS EXTREMOS**")
print("-" * 80)

# Filtrar outliers extremos (>$10K por unidad)
outliers_extremos = df[(df['costo_por_unidad'] > 10000) & valid_mask]

if len(outliers_extremos) > 0:
    print(f"   📊 Total de outliers extremos (>$10K/unidad): {len(outliers_extremos):,} cargos")
    
    # Servicios más frecuentes en outliers extremos
    servicios_outliers = outliers_extremos['area_servicio'].value_counts().head(10)
    print(f"\n   🔝 Top 10 áreas de servicio en outliers extremos:")
    for i, (servicio, count) in enumerate(servicios_outliers.items(), 1):
        costo_promedio = outliers_extremos[outliers_extremos['area_servicio'] == servicio]['costo_por_unidad'].mean()
        print(f"      {i:2d}. {servicio[:50]:<50} | {count:>3,} cargos | Prom: {format_currency(costo_promedio)}")
    
    # Descripciones más frecuentes
    print(f"\n   📝 Top 10 descripciones en outliers extremos:")
    descripciones_outliers = outliers_extremos['descripcion'].value_counts().head(10)
    for i, (desc, count) in enumerate(descripciones_outliers.items(), 1):
        costo_promedio = outliers_extremos[outliers_extremos['descripcion'] == desc]['costo_por_unidad'].mean()
        desc_short = str(desc)[:50] + '..' if len(str(desc)) > 52 else str(desc)
        print(f"      {i:2d}. {desc_short:<50} | {count:>3,} cargos | Prom: {format_currency(costo_promedio)}")

else:
    print("   ℹ️  No se encontraron outliers extremos (>$10K/unidad)")

print(f"\n✅ Análisis de outliers por costo por unidad completado")


💰 DETECCIÓN DE OUTLIERS POR COSTO POR UNIDAD
📊 **Datos válidos para análisis:** 2,399,175 de 2,399,200 registros

🔍 **1. OUTLIERS GLOBALES EN COSTO POR UNIDAD**
------------------------------------------------------------
   • IQR                 :  268,916 cargos outliers (11.21%)
   • Z-Score             :   22,702 cargos outliers ( 0.95%)
   • Modified Z-Score    :  491,530 cargos outliers (20.49%)

🚨 **2. TOP 15 CARGOS CON MAYOR COSTO POR UNIDAD**
------------------------------------------------------------------------------------------------------------------------
PACIENTE     COSTO/UNIDAD    CANTIDAD   COSTO_TOTAL     ORIGEN          AREA_SERVICIO             DESCRIPCION              
------------------------------------------------------------------------------------------------------------------------
607249       $198K           1          $198K           Hospitalización CIRUGÍA NEUMOLOGICA       Trasplante pulmonar      
646096       $198K           1          $198K         

---

## 📋 **RESUMEN EJECUTIVO DE OUTLIERS Y RECOMENDACIONES**

### 🎯 **Hallazgos Críticos del Análisis de Outliers**

<div style="background-color: #f8f9fa; padding: 25px; border-left: 5px solid #007bff; margin: 20px 0; border-radius: 8px;">

#### **⚠️ CASOS CRÍTICOS IDENTIFICADOS**

**1. Outliers de Costo Total por Paciente:**
- **Casos Extremos:** Pacientes con costos >$10M representan alto riesgo financiero
- **Impacto:** Los outliers concentran una proporción significativa del gasto total
- **Patrón:** Alta correlación entre número de cargos y costo total

**2. Outliers de Cantidad/Unidades:**
- **Cargos Masivos:** Registros individuales con >10,000 unidades requieren validación
- **Distribución:** Mayoría de cargos son unitarios, pero los múltiples concentran el volumen
- **Variación por Origen:** Hospitalización muestra mayor frecuencia de cantidades extremas

**3. Outliers de Costo por Unidad:**
- **Servicios Ultra-Costosos:** Cargos >$10K por unidad indican servicios especializados o errores
- **Heterogeneidad:** Gran variabilidad entre orígenes de servicio
- **Concentración:** Pocos servicios específicos representan la mayoría de outliers extremos

#### **🚀 RECOMENDACIONES ESTRATÉGICAS**

**Inmediatas (1-2 semanas):**
1. **Auditoría de Casos Extremos:** Revisar pacientes con costo >$10M
2. **Validación de Cantidades:** Verificar cargos con >10,000 unidades
3. **Control de Calidad:** Implementar alertas para costos unitarios >$50K

**Mediano Plazo (1-3 meses):**
1. **Sistema de Alertas:** Dashboard de monitoreo de outliers en tiempo real
2. **Protocolos de Validación:** Procedimientos automáticos de verificación
3. **Análisis de Causa Raíz:** Investigar orígenes de outliers extremos

**Largo Plazo (3-12 meses):**
1. **Modelo Predictivo:** Sistema de detección temprana de casos atípicos
2. **Optimización de Costos:** Negociación basada en análisis de outliers
3. **Mejora de Procesos:** Rediseño de flujos para prevenir errores

#### **💰 IMPACTO FINANCIERO ESTIMADO**

- **Potencial de Ahorro:** 5-15% del costo total mediante control de outliers
- **Riesgo de Pérdidas:** Casos no detectados pueden representar millones en sobrecostos
- **ROI del Sistema:** Implementación se paga en 6-12 meses con detección mejorada

</div>

---

### 📈 **Próximos Análisis Recomendados**

1. **Análisis Temporal de Outliers:** Tendencias y estacionalidad
2. **Segmentación Clínica:** Outliers por especialidad médica
3. **Análisis de Sobrevivencia:** Duración de tratamientos atípicos
4. **Clustering de Pacientes:** Identificación de perfiles de riesgo
5. **Análisis de Fraude:** Detección de patrones sospechosos

---
